In [37]:
#!pip install agentpy owlready2

In [38]:
import agentpy as ap
from matplotlib import pyplot as plt
import IPython
import random
import ast
from owlready2 import *

In [39]:
# onto = get_ontology("file://onto.owl")

In [40]:
#ONTOLOGIA
#onto.delete()
# with onto:

#     class Entity(Thing):
#         pass

#     class Bot(Entity):
#         pass

#     class Box(Entity):
#         pass

#     class Shelf(Entity):
#         pass

#     class Place(Thing):
#         pass

#     class Position(Thing):
#         pass

#     class has_place(ObjectProperty, FunctionalProperty):
#         domain = [Entity]
#         range = [Place]
#         pass

#     class has_position(DataProperty, FunctionalProperty):
#         domain = [Place]
#         range = [str]
#         pass

In [42]:
class CleanbotAgent(ap.Agent):

    #Razonamiento

    def see(self, e):
        # e = e.strip('()')
        # e = '{' + e + '}'
        # self.per = list(ast.literal_eval(e).items())
        self.per = e
        print(self.per)
        return self.per

    def next(self, per):
        object_key = [key for key, value in per if value == 'Object']
        bot_key = [key for key, value in per if value == 'Bot']
        wall_key = [key for key, value in per if value == 'Wall']
        shelf_key = [key for key, value in per if value == 'Shelf']
        print("Inventory: ", self.inventory)

        # if object_key:
        #     print(f"'Object' is in direction: {object_key}")
        # if wall_key:
        #     print(f"'Wall' is in direction: {wall_key}")
        # if bot_key:
        #     print(f"'Bot' is in direction: {bot_key}")
        # if object_key is None and wall_key is None and bot_key is None:
        #     print("Nothing nearby...")

        for rule in self.rules:
            act = rule(per)
            if act is not None:
                print(f"Rule {rule} returned action: {act.__name__}")
                return act
        print("No rule applied")
        return None

    def action(self, act):
        act()

    #DA RULES

    #Rule 1: if I have an object and I'm in from of a shelf, I drop the object
    def rule_1(self, per):
        validator = [False, False]
        shelf_key = [key for key, value in per if value == 'Shelf']

        if self.inventory == 1:
            validator[0] = True
        for d in shelf_key:
            if d == self.direction:
                validator[1] = True

        if sum(validator) == 2:
            self.inventory = 0
            return self.drop
        pass

    #Rule 2: if I have an object and there's a shelf nearby and I'm not looking at the shelf's direction, I turn to the right
    def rule_2(self, per):
        validator = [False, False, False]
        shelf_key = [key for key, value in per if value == 'Shelf']

        if self.inventory == 1:
            validator[0] = True
        if shelf_key:
            validator[1] = True
        for d in shelf_key:
            if d != self.direction:
                validator[2] = True

        #print("Rule 2:", validator)
        if sum(validator) == 3:
            return self.turn
        pass

    #Rule 3: if I have an object and there's not a shelf nearby and there's something in front of me, I turn to the right
    def rule_3(self, per):
        validator = [False, False, False, False]
        shelf_key = [key for key, value in per if value == 'Shelf']
        object_key = [key for key, value in per if value == 'Object']
        wall_key = [key for key, value in per if value == 'Wall']

        if self.inventory == 1:
            validator[0] = True
        if not shelf_key:
            validator[1] = True
        if object_key == self.direction:
            validator[2] = True
        if wall_key == self.direction:
            validator[3] = True

        print("Rule 3:", validator)
        if sum(validator) == 4:
            return self.turn
        pass

    #Rule 4: if I have an object and there's not a shelf nearby and there's nothing in front of me, I move forward
    def rule_4(self, per):
        validator = [False, False, False]
        shelf_key = [key for key, value in per if value == 'Shelf']
        object_key = [key for key, value in per if value == 'Object']
        wall_key = [key for key, value in per if value == 'Wall']

        if self.inventory == 1:
            validator[0] = True
        if not shelf_key:
            validator[1] = True
        if self.direction not in object_key and self.direction not in wall_key:
            validator[2] = True

        if sum(validator) == 3:
            return self.move
        pass

    #Rule 5: if I don't have an object and I see an object nearby and it's in front of me I grab it
    def rule_5(self, per):
        validator = [False, False, False]
        object_key = [key for key, value in per if value == 'Object']

        if self.inventory == 0:
            validator[0] = True
        if object_key:
            validator[1] = True
        for d in object_key:
            if d == self.direction:
                validator[2] = True
        if sum(validator) == 3:
            return self.grab


    #Rule 6: if I don't have an object and I see an object nearby and it's not in front of me I turn to the right
    def rule_6(self, per):
        validator = [False, False, False]
        object_key = [key for key, value in per if value == 'Object']

        if self.inventory == 0:
            validator[0] = True
        if object_key:
            validator[1] = True
        for d in object_key:
            if d != self.direction:
                validator[2] = True

        if sum(validator) == 3:
            return self.turn

    #Rule 7: if I see an object nearby and it's not in front of me I turn towards it
    def rule_7(self, per):
        validator = [False, False, False, False]
        object_key = [key for key, value in per if value == 'Object']
        wall_key = [key for key, value in per if value == 'Wall']
        shelf_key = [key for key, value in per if value == 'Shelf']

        if self.inventory == 0:
            validator[0] = True
        if not object_key:
            validator[1] = True
        if self.direction not in wall_key:
            validator[2] = True
        if self.direction not in shelf_key:
            validator[3] = True
        #print(f"Rule 7 validators: {validator}")
        if sum(validator) == 4:
            #print("Rule 7 conditions met, returning move action")
            return self.move
        return None

    #Rule 8: if I don't have an object and I don't see an object nearby and I have a wall in front of me or a shelf in front of me, I turn
    def rule_8(self, per):
        validator = [False, False, False]
        object_key = [key for key, value in per if value == 'Object']
        wall_key = [key for key, value in per if value == 'Wall']
        shelf_key = [key for key, value in per if value == 'Shelf']

        if self.inventory == 0:
            validator[0] = True
        if not object_key:
            validator[1] = True
        if self.direction in wall_key or self.direction in shelf_key:
            validator[2] = True
        if sum(validator) == 3:
            return self.turn
        return None

    #Simulación de Agente

    def setup(self):
        self.agentType = 0
        self.direction = "N"
        self.dir = (-1, 0)
        self.inventory = 0

        self.rules = (
            self.rule_1,
            self.rule_2,
            self.rule_3,
            self.rule_4,
            self.rule_5,
            self.rule_6,
            self.rule_7,
            self.rule_8
        )

        self.actions = (
            self.move,
            self.turn,
            self.grab,
            self.drop,
        )

        #erase this later
        self.first_step = True


    def step(self):
        if self.first_step:
            self.idle()
            self.first_step = False
        else:
            self.e = self.get_env(self.model.grid)
            self.per = self.see(self.e)
            self.act = self.next(self.per)
            if self.act is not None:
                self.action(self.act)
            print(f"Direction: {self.direction}")
            print(f"Action: {self.act.__name__ if self.act else 'None'}")





    #ACTIONS

    def move(self):
        if self.direction == 'N':
            self.model.grid.move_by(self, (-1, 0))
        if self.direction == 'S':
            self.model.grid.move_by(self, (1, 0))
        if self.direction == 'E':
            self.model.grid.move_by(self, (0, 1))
        if self.direction == 'W':
            self.model.grid.move_by(self, (0, -1))
        print(f"Moved to new position: {self.model.grid.positions[self]}")
        pass

    def turn(self):
        if self.direction == "N":
            self.dir = (0,1) #Hacia Este
            self.direction = "E"
        elif self.direction == "E":
            self.dir = (1,0)  #Hacia Sur
            self.direction = "S"
        elif self.direction == "S":
            self.dir = (0,-1) #Hacia Oeste
            self.direction = "W"
        elif self.direction == "W":
            self.dir = (-1,0) #Hacia Norte
            self.direction = "N"
        pass

    def grab(self):
        print(f"Attempting to grab. Agent position: {self.model.grid.positions[self]}")
        front_pos = self.find_boxpos()
        print(f"Looking for object at: {front_pos}")
        for box in self.model.grid.agents:
            box_pos = self.model.grid.positions[box]
            if box.agentType == 1 and box_pos == front_pos:
                try:
                    self.model.grid.remove_agents(box)
                    print("Box removed successfully")
                    self.inventory += 1  # Don't forget to update the inventory
                except Exception as e:
                    print(f"Error removing box: {e}")
                break
            else:
                pass

    def drop(self):
        self.inventory == 0
        pass

    def find_boxpos(self):
        current_pos = self.model.grid.positions[self]
        if self.direction == 'N':
            return (current_pos[0] - 1, current_pos[1])
        elif self.direction == 'S':
            return (current_pos[0] + 1, current_pos[1])
        elif self.direction == 'E':
            return (current_pos[0], current_pos[1] + 1)
        elif self.direction == 'W':
            return (current_pos[0], current_pos[1] - 1)
        else:
            raise ValueError("Invalid direction")

    def get_env(self, env):
        neighbors = env.neighbors(self, 1)
        current_position = env.positions[self]
        self.e = self.transform_neighbors(neighbors, current_position)
        if not self.e:
            self.e = [('N', '0'), ('S', '0'), ('W', '0'), ('E', '0')]
        return self.e

    def transform_neighbors(self, neighbors, current_position):
        directions = ['N', 'S', 'W', 'E']
        result = {d: '0' for d in directions}  # Initialize all directions with '0'

        for direction in directions:
            if direction == 'N':
                position = (current_position[0] - 1, current_position[1])
            elif direction == 'S':
                position = (current_position[0] + 1, current_position[1])
            elif direction == 'W':
                position = (current_position[0], current_position[1] - 1)
            elif direction == 'E':
                position = (current_position[0], current_position[1] + 1)

            if self.is_wall(position):
                result[direction] = "Wall"
            else:
                cell_content = self.model.grid.agents[position]
                if cell_content:
                    # Use next() to get the first agent if there is one
                    agent = next(iter(cell_content), None)
                    if agent and hasattr(agent, 'agentType'):
                        if str(agent.agentType) == "0":
                            result[direction] = "Bot"
                        elif str(agent.agentType) == "1":
                            result[direction] = "Object"
                        elif str(agent.agentType) == "2":
                            result[direction] = "Shelf"
                        else:
                            result[direction] = "0"
                    else:
                        result[direction] = "0"
                else:
                    result[direction] = "0"  # Empty cell

        return list(result.items())

    def is_wall(self, position):
        x, y = position
        return x < 0 or x >= self.model.p.M or y < 0 or y >= self.model.p.N


    def idle(self):
        pass



In [43]:
class BoxAgent(ap.Agent):

    def setup(self):
        self.agentType = 1
        self.first_step = True
        self.pos = None
        pass

    def step(self):
        if self.first_step:
            self.pos = self.model.grid.positions[self]
            self.first_step = False
        pass

    def update(self):
        pass

    def end(self):
        pass

In [44]:
class ShelfAgent(ap.Agent):

    def setup(self):
        self.agentType = 2
        pass

    def step(self):
        pass

    def update(self):
        pass

    def end(self):
        pass

In [45]:
class WarehouseModel(ap.Model):
    def setup(self):
        self.cleanbots = ap.AgentList(self, self.p.cleanbots, CleanbotAgent)
        self.boxes = ap.AgentList(self, self.p.boxes, BoxAgent)
        self.shelves = ap.AgentList(self, self.p.shelves, ShelfAgent)


        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
        self.grid.add_agents(self.shelves, positions=(self.p.wpos), random=False, empty=True)
        self.grid.add_agents(self.cleanbots, random=True, empty=True)
        self.grid.add_agents(self.boxes, random=True, empty=True)
        pass

    def step(self):
        self.cleanbots.step()
        self.boxes.step()
        self.shelves.step()
        pass

    def update(self):
        pass

    def end(self):
        pass



In [46]:
parameters = {
    'M': 20,
    'N': 14,
    'steps': 20,
    'cleanbots': 1,
    'boxes': 30,
    'shelves': 24,
    'wpos': ((0,0), (0,1), (0, 2), (1, 0), (1, 1), (1, 2),
             (3, 13), (4, 13), (5, 13), (3, 12), (4, 12), (5, 12),
             (17, 12), (18, 12), (19, 12), (17, 13), (18, 13), (19, 13),
             (10, 0), (11, 0), (12, 0), (10, 1), (11, 1), (12, 1))
}

# model = WarehouseModel(parameters)
# results = model.run()

In [47]:
#ANIMATION
def animation_plot(model, ax):
    # Definición de atributo para tipo de agente
    agent_type_grid = model.grid.attr_grid('agentType')
    # Definición de gráfico con colores (de acuerdo al tipo de agente)
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    # Definición de título del gráfico
    ax.set_title(f"Warehouse \n Time-step: {model.t}, "
                 f"Boxes: {0}")


In [48]:
#SIMULATION

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = WarehouseModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())


[('N', '0'), ('S', '0'), ('W', '0'), ('E', '0')]
Inventory:  0
Rule 3: [False, True, False, False]
Rule <bound method CleanbotAgent.rule_7 of CleanbotAgent (Obj 1)> returned action: move
Moved to new position: (12, 5)
Direction: N
Action: move
[('N', '0'), ('S', '0'), ('W', 'Object'), ('E', '0')]
Inventory:  0
Rule 3: [False, True, False, False]
Rule <bound method CleanbotAgent.rule_6 of CleanbotAgent (Obj 1)> returned action: turn
Direction: E
Action: turn
[('N', '0'), ('S', '0'), ('W', 'Object'), ('E', '0')]
Inventory:  0
Rule 3: [False, True, False, False]
Rule <bound method CleanbotAgent.rule_6 of CleanbotAgent (Obj 1)> returned action: turn
Direction: S
Action: turn
[('N', '0'), ('S', '0'), ('W', 'Object'), ('E', '0')]
Inventory:  0
Rule 3: [False, True, False, False]
Rule <bound method CleanbotAgent.rule_6 of CleanbotAgent (Obj 1)> returned action: turn
Direction: W
Action: turn
[('N', '0'), ('S', '0'), ('W', 'Object'), ('E', '0')]
Inventory:  0
Rule 3: [False, True, False, False